# Imports

In [1]:
import numpy as np

# Add these lines if you clone from https://github.com/fdalvi/NeuroX
# import sys
# sys.path.append("/path/to/NeuroX")

import sys
sys.path.append("..")
import neurox.data.extraction.transformers_extractor as transformers_extractor
import neurox.data.loader as data_loader

import neurox.interpretation.ablation as ablation
import neurox.interpretation.linear_probe as linear_probe
import neurox.interpretation.utils as utils

# Data

In [2]:
!cat test.in

This is a TEST SENNNTENCE.
With some UNKN*EJOWN words!!
This is a TEST SENNNTENCE.
With some UNKN*EJOWN words!!
This is a TEST SENNNTENCE.
With some UNKN*EJOWN words!!
This is a TEST SENNNTENCE.
With some UNKN*EJOWN words!!
This is a TEST SENNNTENCE.
With some UNKN*EJOWN words!!

In [3]:
!cat test.label

LABEL1 LABEL2 LABEL3 LABEL1 LABEL3
LABEL3 LABEL1 LABEL3 LABEL2
LABEL1 LABEL2 LABEL3 LABEL1 LABEL3
LABEL3 LABEL1 LABEL3 LABEL2
LABEL1 LABEL2 LABEL3 LABEL1 LABEL3
LABEL3 LABEL1 LABEL3 LABEL2
LABEL1 LABEL2 LABEL3 LABEL1 LABEL3
LABEL3 LABEL1 LABEL3 LABEL2
LABEL1 LABEL2 LABEL3 LABEL1 LABEL3
LABEL3 LABEL1 LABEL3 LABEL2

# Extract Representations

In [4]:
transformers_extractor.extract_representations('bert-base-uncased',
    'test.in',
    'activations.json',
    aggregation="average" #last, first
)

Loading model: bert-base-uncased


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Reading input corpus
Preparing output file
Extracting representations from model
Sentence         : "This is a TEST SENNNTENCE."
Original    (005): ['This', 'is', 'a', 'TEST', 'SENNNTENCE.']
Tokenized   (011): ['[CLS]', 'this', 'is', 'a', 'test', 'sen', '##nn', '##ten', '##ce', '.', '[SEP]']
Filtered   (009): ['this', 'is', 'a', 'test', 'sen', '##nn', '##ten', '##ce', '.']
Detokenized (005): ['this', 'is', 'a', 'test', 'sen##nn##ten##ce.']
Counter: 9
Hidden states:  (13, 5, 768)
# Extracted words:  5
Sentence         : "With some UNKN*EJOWN words!!"
Original    (004): ['With', 'some', 'UNKN*EJOWN', 'words!!']
Tokenized   (014): ['[CLS]', 'with', 'some', 'un', '##k', '##n', '*', 'e', '##jo', '##wn', 'words', '!', '!', '[SEP]']
Filtered   (012): ['with', 'some', 'un', '##k', '##n', '*', 'e', '##jo', '##wn', 'words', '!', '!']
Detokenized (004): ['with', 'some', 'un##k##n*e##jo##wn', 'words!!']
Counter: 12
Hidden states:  (13, 4, 768)
# Extracted words:  4
Sentence         : "This is a TE

In [5]:
!cut -c1-100 < activations.json

{"linex_index": 0, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.6
{"linex_index": 1, "features": [{"token": "with", "layers": [{"index": 0, "values": [-0.36680433, 0.
{"linex_index": 2, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.6
{"linex_index": 3, "features": [{"token": "with", "layers": [{"index": 0, "values": [-0.36680433, 0.
{"linex_index": 4, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.6
{"linex_index": 5, "features": [{"token": "with", "layers": [{"index": 0, "values": [-0.36680433, 0.
{"linex_index": 6, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.6
{"linex_index": 7, "features": [{"token": "with", "layers": [{"index": 0, "values": [-0.36680433, 0.
{"linex_index": 8, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.6
{"linex_index": 9, "features": [{"token": "with", "layers": [{"index": 0, "values": [-0.366

# Prepare Data

In [6]:
activations, num_layers = data_loader.load_activations('activations.json', 768)

Loading json activations from activations.json...
10 13.0


In [7]:
activations

[array([[-0.6485135 ,  0.67392403, -0.09324985, ..., -0.45096967,
          0.46059093,  0.93923897],
        [-0.62703037, -0.06331295, -0.31427881, ..., -0.28688025,
          0.12916157,  1.09747875],
        [ 0.39637804,  0.31565616,  0.02008923, ..., -0.18841645,
          0.18347587,  1.51162875],
        [ 0.60103226, -0.69701707, -0.2000732 , ..., -0.51888162,
         -0.24854559,  0.26434106],
        [-0.24404168, -0.66105956, -0.27814269, ..., -0.15310135,
         -0.08717965,  0.1507422 ]]),
 array([[-0.36680433,  0.361595  ,  0.22941463, ..., -0.44721714,
         -0.04346137,  0.18839234],
        [-0.60318661,  0.53118145, -0.50209051, ..., -0.54392618,
         -0.31075817,  0.4812808 ],
        [-0.39689821, -0.45037764,  0.05965039, ..., -0.45414358,
         -0.26728544,  0.25421724],
        [ 0.32986507,  0.2577436 , -0.37915197, ...,  0.03252593,
         -0.43623337, -0.20786768]]),
 array([[-0.6485135 ,  0.67392403, -0.09324985, ..., -0.45096967,
          0.

In [8]:
activations[0].shape # 9984 -> 768 * 13

(5, 9984)

In [9]:
activations[1].shape

(4, 9984)

In [10]:
# load_data also does sanity checks for parallelism between tokens, labels and activations
tokens = data_loader.load_data('test.in',
                               'test.label',
                               activations,
                               512 # max_sent_l
                              )

In [11]:
tokens

{'source': [['This', 'is', 'a', 'TEST', 'SENNNTENCE.'],
  ['With', 'some', 'UNKN*EJOWN', 'words!!'],
  ['This', 'is', 'a', 'TEST', 'SENNNTENCE.'],
  ['With', 'some', 'UNKN*EJOWN', 'words!!'],
  ['This', 'is', 'a', 'TEST', 'SENNNTENCE.'],
  ['With', 'some', 'UNKN*EJOWN', 'words!!'],
  ['This', 'is', 'a', 'TEST', 'SENNNTENCE.'],
  ['With', 'some', 'UNKN*EJOWN', 'words!!'],
  ['This', 'is', 'a', 'TEST', 'SENNNTENCE.'],
  ['With', 'some', 'UNKN*EJOWN', 'words!!']],
 'target': [['LABEL1', 'LABEL2', 'LABEL3', 'LABEL1', 'LABEL3'],
  ['LABEL3', 'LABEL1', 'LABEL3', 'LABEL2'],
  ['LABEL1', 'LABEL2', 'LABEL3', 'LABEL1', 'LABEL3'],
  ['LABEL3', 'LABEL1', 'LABEL3', 'LABEL2'],
  ['LABEL1', 'LABEL2', 'LABEL3', 'LABEL1', 'LABEL3'],
  ['LABEL3', 'LABEL1', 'LABEL3', 'LABEL2'],
  ['LABEL1', 'LABEL2', 'LABEL3', 'LABEL1', 'LABEL3'],
  ['LABEL3', 'LABEL1', 'LABEL3', 'LABEL2'],
  ['LABEL1', 'LABEL2', 'LABEL3', 'LABEL1', 'LABEL3'],
  ['LABEL3', 'LABEL1', 'LABEL3', 'LABEL2']]}

In [12]:
X, y, mapping = utils.create_tensors(tokens, activations, 'LABEL1')
label2idx, idx2label, src2idx, idx2src = mapping
# X_dev, y_dev, mapping = utils.create_tensors(dev_tokens, dev_activations, 'LABEL1')

Number of tokens:  45
length of source dictionary:  9
length of target dictionary:  3
45
Total instances: 45
['TEST', 'UNKN*EJOWN', 'SENNNTENCE.', 'a', 'With', 'words!!', 'some', 'is', 'This']


In [13]:
X[0] #9984

array([-0.6485135 ,  0.673924  , -0.09324985, ..., -0.45096967,
        0.46059093,  0.93923897], dtype=float32)

# Train Probing Classifier

In [14]:
probe = linear_probe.train_logistic_regression_probe(X, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 45
Number of classes: 3


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0421


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0086


...

Epoch: [8/10], Loss: 0.0078


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.0079


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0078


# Evaluate Model

In [15]:
linear_probe.evaluate_probe(probe, X, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 1.00


{'__OVERALL__': 1.0, 'LABEL3': 1.0, 'LABEL1': 1.0, 'LABEL2': 1.0}

# Layerwise probing

In [16]:
# [layer0_neuron0, layer0_neuron1, layer0_neuron2 ...., layer1_neuron0, layer1_neuron1,....., layer13_neuron768]
# [768,768,770...1536] <- for layer 1
num_neurons_per_layer = 768
layer_1_X = ablation.filter_activations_keep_neurons(X, np.arange(num_neurons_per_layer, num_neurons_per_layer*2))

In [17]:
layer_1_X.shape

(45, 768)

In [18]:
probe_layer_1 = linear_probe.train_logistic_regression_probe(layer_1_X, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 45
Number of classes: 3


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0573


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0437


...

Epoch: [8/10], Loss: 0.0098


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.0080


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0068


In [19]:
linear_probe.evaluate_probe(probe_layer_1, layer_1_X, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 1.00


{'__OVERALL__': 1.0, 'LABEL3': 1.0, 'LABEL1': 1.0, 'LABEL2': 1.0}

# Get Neuron Ranking

In [20]:
ordering, cutoffs = linear_probe.get_neuron_ordering(probe, label2idx)

  0%|          | 0/101 [00:00<?, ?it/s]

In [21]:
ordering[:10]

[4928, 6058, 3122, 6659, 516, 1284, 2308, 3847, 358, 2572]

# Train on top N neurons

In [22]:
X_selected = ablation.filter_activations_keep_neurons(X, ordering[:100])

In [23]:
X_selected.shape

(45, 100)

In [24]:
probe_selected = linear_probe.train_logistic_regression_probe(X_selected, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 45
Number of classes: 3


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.0518


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0499


...

Epoch: [8/10], Loss: 0.0395


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.0380


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0365


In [25]:
linear_probe.evaluate_probe(probe_selected, X_selected, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.89


{'__OVERALL__': 0.8888888888888888,
 'LABEL3': 1.0,
 'LABEL1': 0.6666666666666666,
 'LABEL2': 1.0}

# Further analysis

In [26]:
top_neurons = linear_probe.get_top_neurons(probe, 0.01, label2idx)

In [27]:
top_neurons

(array([6659,  516, 2572, 4115, 6176, 1059, 5156,   40, 3630, 2095, 3122,
        6194, 1594, 2108, 2109, 7741, 2631, 8777,   78, 6243, 8292, 7269,
        3172,  615, 2664, 2665, 3179, 6763, 4717, 5230, 7787, 1139, 4730,
        7805, 6784, 7812, 6279, 6284, 2711, 5787, 5280, 5797, 3751, 2225,
        9416, 4298, 4299, 5836, 6348, 2255, 7891, 6870, 7910, 3814, 4340,
        7415, 1284, 2308, 3847,  278, 3359, 3360, 1317, 4901, 3880, 9001,
        4403, 3382,  317, 4928, 4423, 5977, 9568, 6500,  358, 8055, 5508,
        4997, 7569, 9105, 8081,  407, 3483, 7580, 1442, 4517, 1961, 6058,
         938, 8111,  969, 3532, 6101,  471, 2521, 4574, 4581, 3045, 1523,
        2037, 1016, 9215]),
 {'LABEL3': array([6058, 8292, 4403, 1059, 2521, 2711, 6176, 1961, 6101, 2109, 8055,
          471, 5977, 1442, 7812, 2095, 6870, 9001, 7569, 3359, 7910,  278,
          938,  615, 3814, 5787, 6243, 1523,   40, 3483, 5280, 3179, 4717,
         2255]),
  'LABEL1': array([3122, 7580, 1139, 7269, 5797, 7805,

In [28]:
X_zeroed = ablation.zero_out_activations_keep_neurons(X, ordering[:1000])
print(X_zeroed.shape)
print(X_zeroed)
linear_probe.evaluate_probe(probe, X_zeroed, y, idx_to_class=idx2label)

(45, 9984)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 1.00


{'__OVERALL__': 1.0, 'LABEL3': 1.0, 'LABEL1': 1.0, 'LABEL2': 1.0}

In [29]:
import neurox.analysis.visualization as visualization
visualization.visualize_activations("This is a TEST SENNNTENCE.", activations[0][:, ordering[0]])